In [1]:
from sqlalchemy.orm import registry

mapper_registry = registry()
Base = mapper_registry.generate_base()

from sqlalchemy import select
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey

from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session

from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)


class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return (
            f"User(id={self.id!r},"
            f" name={self.name!r},"
            f" fullname={self.fullname!r})"
        )


class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}," f" email_address=" f"{self.email_address!r})"

In [2]:
mapper_registry.metadata.create_all(engine)

2022-04-03 18:50:21,608 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:50:21,608 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-03 18:50:21,610 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:50:21,611 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-04-03 18:50:21,612 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:50:21,614 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-03 18:50:21,615 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:50:21,616 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-04-03 18:50:21,618 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:50:21,619 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-03 18:50:21,620 INFO sqlalchemy.engine.Engine [no key 0.00055s] ()
2022-04-03 18:50:21,621 INFO sqlalchemy.engine.Engine 
C

In [10]:
stmt = insert(User).\
    values(name='spongebob', 
    fullname="Spongebob Squarepants")

In [12]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [14]:
compiled = stmt.compile()

In [15]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [16]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-04-03 18:56:03,145 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:56:03,147 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-03 18:56:03,150 INFO sqlalchemy.engine.Engine [generated in 0.00503s] ('spongebob', 'Spongebob Squarepants')
2022-04-03 18:56:03,151 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
with engine.connect() as conn:
    result = conn.execute(
        insert(User),
        [
            {"name": "sandy", 
            "fullname": "Sandy Cheeks"},
            {"name": "patrick", 
            "fullname": "Patrick Star"}
        ]
    )
    conn.commit()

2022-04-03 18:56:06,646 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:56:06,648 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-03 18:56:06,650 INFO sqlalchemy.engine.Engine [generated in 0.00421s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-04-03 18:56:06,650 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
User.id

In [5]:
User.name

In [7]:
from sqlalchemy import select, bindparam

scalar_subq = (
    select(User.id)
    .where(User.name == bindparam("username"))
    .scalar_subquery()
)

from sqlalchemy import insert
with engine.connect() as conn:
    result = conn.execute(
        insert(Address).values(user_id=scalar_subq),
        [
            {"username": "spongebob", "email_address": "spongebob@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2022-04-03 18:51:37,133 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:51:37,134 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, (SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?))
2022-04-03 18:51:37,135 INFO sqlalchemy.engine.Engine [generated in 0.00194s] (('spongebob@sqlalchemy.org', 'spongebob'), ('sandy@sqlalchemy.org', 'sandy'), ('sandy@squirrelpower.org', 'sandy'))
2022-04-03 18:51:37,136 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
stmt = select(User).where(User.name == 'spongebob')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-04-03 18:56:12,552 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:56:12,553 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-03 18:56:12,554 INFO sqlalchemy.engine.Engine [cached since 271.4s ago] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-04-03 18:56:12,555 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
import sqlalchemy
sqlalchemy.__version__

'1.4.32'